In [1]:
import pandas as pd
import numpy as np
    
df = pd.read_csv('data/john.csv', parse_dates=['date', 'arrival', 'departure'], infer_datetime_format=True, dayfirst=True)

df = df.set_index('date')
df = df.sort_index()


df['water_level'] = pd.to_numeric(df['water_level'].str.replace(',', '.'))
df['time_in_shower_since_departure'] = pd.to_timedelta(df['time_in_shower_since_departure'])

df = df.resample('1Min').asfreq().resample('5min').last()

df['date'] = df.index
df['arrival'] = df['arrival'].ffill()
df['departure'] = df['departure'].ffill()
df['time_from_departure'] = df['date'] - df['departure']
df['time_to_arrival'] = df['arrival'] - df['date']

df

,status,timezone,number_of_passengers,departure,arrival,time_in_shower_since_departure,water_level,date,time_from_departure,time_to_arrival
date,,,,,,,,,,
2017-06-26 17:15:00,IN_FLIGHT,+02:00,3.0,2017-06-26 17:15:00,2017-06-27 05:10:00,0 days,1.00,2017-06-26 17:15:00,00:00:00,11:55:00
2017-06-26 17:20:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:20:00,00:05:00,11:50:00
2017-06-26 17:25:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:25:00,00:10:00,11:45:00
2017-06-26 17:30:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:30:00,00:15:00,11:40:00
2017-06-26 17:35:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:35:00,00:20:00,11:35:00
2017-06-26 17:40:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:40:00,00:25:00,11:30:00
2017-06-26 17:45:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,0.95,2017-06-26 17:45:00,00:30:00,11:25:00
2017-06-26 17:50:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:50:00,00:35:00,11:20:00
2017-06-26 17:55:00,NaN,NaN,NaN,2017-06-26 17:15:00,2017-06-27 05:10:00,NaT,NaN,2017-06-26 17:55:00,00:40:00,11:15:00


In [2]:
from os import environ
from dotenv import load_dotenv, find_dotenv
import craftai.pandas

load_dotenv(find_dotenv())

# Create a craft ai client
client = craftai.pandas.Client({
    "token": environ.get("CRAFT_TOKEN")
})

In [3]:
agent_cfg = {
    "context": {
        "timezone": {
            "type": "timezone"
        },
        "time": {
            "type": "time_of_day"
        },
        "day": {
            "type": "day_of_week"
        },
        "status": {
            "type": "enum"
        },
        "number_of_passengers": {
            "type": "continuous"
        },
#        "time_from_departure": {
#            "type": "continuous"
#        },
        "time_to_arrival": {
            "type": "continuous"
        },
#        "time_in_shower_since_departure": {
#            "type": "continuous"
#        },
        "water_level": {
            "type": "continuous"
        }
    },
    "output": ["water_level"],
    "time_quantum": 60 * 5
}

client.delete_agent('john')
client.create_agent(agent_cfg, 'john')

# Prepare the operations
operations = pd.DataFrame(df[[
    'timezone', 
    'status', 
    'number_of_passengers', 
#    'time_from_departure', 
    'time_to_arrival', 
#    'time_in_shower_since_departure', 
    'water_level'
]])
#operations['time_from_departure'] = operations['time_from_departure'].dt.seconds / 60
operations['time_to_arrival'] = operations['time_to_arrival'].dt.seconds / 60
#operations['time_in_shower_since_departure'] = operations['time_in_shower_since_departure'].dt.seconds / 60
client.add_operations('john', operations)

{'message': 'Successfully added 8307 operation(s) to the agent "cloderic/hackathon_falcon/john" context.'}

In [4]:
tree = client.get_decision_tree('john', 1500989100)

context = {
    "timezone": "+08:00",
    "status": "IN_FLIGHT",
    "number_of_passengers": 2,
    "day": 0,
    "time": 12,
    "time_to_arrival": 2 * 60,
}
client.decide(tree, context)

{'_version': '1.1.0',
 'context': {'day': 0,
  'number_of_passengers': 2,
  'status': 'IN_FLIGHT',
  'time': 12,
  'time_to_arrival': 120,
  'timezone': '+08:00'},
 'output': {'water_level': {'confidence': 0.9402450323104858,
   'decision_rules': [{'operand': 'IN_FLIGHT',
     'operator': 'is',
     'property': 'status'},
    {'operand': [10.166667, 23.25], 'operator': '[in[', 'property': 'time'},
    {'operand': 541, 'operator': '<', 'property': 'time_to_arrival'},
    {'operand': [10.166667, 17.833334],
     'operator': '[in[',
     'property': 'time'}],
   'predicted_value': 0.8567381,
   'standard_deviation': 0.05055386}}}